In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [4]:
chunksize = 5 * 10 ** 5
filename = 'matches_small.csv'
print(pd.read_csv(filename, nrows=5))

     match_id  match_seq_num radiant_win  start_time  duration  \
0  2304340261     2019317886           t  1461013929      1701   
1  2304335744     2019317899           t  1461013382      2299   
2  2304324185     2019317908           t  1461012679      3291   
3  2304339409     2019317931           f  1461013865      2192   
4  2304329004     2019317805           t  1461012987      2989   

   tower_status_radiant  tower_status_dire  barracks_status_radiant  \
0                  1975                  4                       63   
1                  1983                  0                       63   
2                  1584                  0                       60   
3                     0               1982                        0   
4                     4                  0                        3   

   barracks_status_dire  cluster  ...  engine  picks_bans  parse_status  chat  \
0                     3      155  ...       1         NaN             3   NaN   
1             

In [7]:
#run this once to create database
chunksize = 5 * 10 ** 5
filename = 'matches_small.csv'
csv_database = create_engine('sqlite:///csv_database.db')
for chunk in pd.read_csv(filename, chunksize=chunksize):
    i = 0
    j = 1
    for df in pd.read_csv(filename, chunksize=chunksize, iterator=True):
          df = df.rename(columns={c: c.replace(' ', '') for c in df.columns}) 
          df.index += j
          i+=1
          df.to_sql('table', csv_database, if_exists='append')
          j = df.index[-1] + 1

In [2]:
#run this every time
csv_database = create_engine('sqlite:///csv_database.db')
df = pd.read_sql_query("SELECT * FROM 'table'LIMIT 1000000", csv_database)

In [3]:
for col in df.columns: 
    print(col) 
df.shape

index
match_id
match_seq_num
radiant_win
start_time
duration
tower_status_radiant
tower_status_dire
barracks_status_radiant
barracks_status_dire
cluster
first_blood_time
lobby_type
human_players
leagueid
positive_votes
negative_votes
game_mode
engine
picks_bans
parse_status
chat
objectives
radiant_gold_adv
radiant_xp_adv
teamfights
version
pgroup


(1000000, 28)

In [4]:
#cleaning out columns that aren't needed are complicated
# website as reference https://wiki.teamfortress.com/wiki/WebAPI/GetMatchDetails#Tower_Status
df = df.drop(['index'], axis=1)
df = df.drop(['match_id'], axis=1) #match id isn't useful for this type of data
df =df.drop(['match_seq_num'], axis=1) #match_seq_num isn't important
#radiant_win is used for victory (what we use to predict)
df = df.drop(['start_time'], axis=1) #seems random
#duration seems useful
df = df.drop(["cluster"],axis=1) #not important
df = df.drop(["leagueid"],axis=1) #not important 
df = df.drop(["positive_votes"],axis=1) #not important
df = df.drop(["negative_votes"],axis=1) #not important
df = df.drop(["engine"],axis=1) #not important
df = df.drop(["parse_status"],axis=1) #not important
df = df.drop(["chat"],axis=1) #not important
df = df.drop(["objectives"],axis=1) #other things can represent this, such as tower status and barracks status
df = df.drop(["version"],axis=1) # not important
df = df.drop(["pgroup"],axis=1) # not important

df = df.drop(["picks_bans"],axis=1) #pick_bans seems complicated (does hero order and all that)

In [5]:
for col in df.columns: 
    print(col) 
df.shape
df

radiant_win
duration
tower_status_radiant
tower_status_dire
barracks_status_radiant
barracks_status_dire
first_blood_time
lobby_type
human_players
game_mode
radiant_gold_adv
radiant_xp_adv
teamfights


,radiant_win,duration,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire,first_blood_time,lobby_type,human_players,game_mode,radiant_gold_adv,radiant_xp_adv,teamfights
0,t,1701,1975,4,63,3,100,0,10,1,None,None,None
1,t,2299,1983,0,63,0,127,0,10,18,None,None,None
2,t,3291,1584,0,60,0,196,0,10,1,None,None,None
3,f,2192,0,1982,0,63,44,0,10,5,None,None,None
4,t,2989,4,0,3,48,104,0,10,1,None,None,None
5,f,2586,0,2046,0,63,111,0,10,1,None,None,None
6,t,2759,1974,260,63,51,142,0,10,4,None,None,None
7,t,1485,2046,4,63,3,138,7,10,22,None,None,None
8,t,2734,1972,0,63,0,320,0,10,1,None,None,None
9,f,3500,0,1958,0,63,10,7,10,22,None,None,None


In [115]:
#radiant tower status
radiant_ancient_bottom = [] 
radiant_ancient_top = [] 
radiant_bot_tier3 = [] 
radiant_bot_tier2  = [] 
radiant_bot_tier1  = [] 
radiant_mid_tier3 = [] 
radiant_mid_tier2  = [] 
radiant_mid_tier1  = [] 
radiant_top_tier3 = [] 
radiant_top_tier2  = [] 
radiant_top_tier1  = [] 

for z in df["tower_status_radiant"]:
    element = format(z,"016b")
    if element[5]=='1':
        radiant_ancient_bottom.append(1)
    else:
        radiant_ancient_bottom.append(0)
    if element[6]=='1':
        radiant_ancient_top.append(1)
    else:
        radiant_ancient_top.append(0)
    if element[7]=='1':
        radiant_bot_tier3.append(1)
    else:
        radiant_bot_tier3.append(0)
    if element[8]=='1':
        radiant_bot_tier2.append(1)
    else:
        radiant_bot_tier2.append(0) 
    if element[9]=='1':
        radiant_bot_tier1.append(1)
    else:
        radiant_bot_tier1.append(0) 
    if element[10]=='1':
        radiant_mid_tier3.append(1)
    else:
        radiant_mid_tier3.append(0) 
    if element[11]=='1':
        radiant_mid_tier2.append(1)
    else:
        radiant_mid_tier2.append(0)
    if element[12]=='1':
        radiant_mid_tier1.append(1)
    else:
        radiant_mid_tier1.append(0)
    if element[13]=='1':
        radiant_top_tier3.append(1)
    else:
        radiant_top_tier3.append(0)
    if element[14]=='1':
        radiant_top_tier2.append(1)
    else:
        radiant_top_tier2.append(0)
    if element[15]=='1':
        radiant_top_tier1.append(1)
    else:
        radiant_top_tier1.append(0)

df["radiant_ancient_bottom"] = radiant_ancient_bottom 
df["radiant_ancient_top"] = radiant_ancient_top 
df["radiant_bot_tier3"] = radiant_bot_tier3 
df["radiant_bot_tier2"] = radiant_bot_tier2  
df["radiant_bot_tier1"] = radiant_bot_tier1   
df["radiant_mid_tier3"] = radiant_mid_tier3  
df["radiant_mid_tier2"] = radiant_mid_tier2   
df["radiant_mid_tier1"] = radiant_mid_tier1  
df["radiant_top_tier3"] = radiant_top_tier3 
df["radiant_top_tier2"] =radiant_top_tier2  
df["radiant_top_tier1"] =radiant_top_tier1  

In [116]:
#dire tower status
dire_ancient_bottom = [] 
dire_ancient_top = [] 
dire_bot_tier3 = [] 
dire_bot_tier2  = [] 
dire_bot_tier1  = [] 
dire_mid_tier3 = [] 
dire_mid_tier2  = [] 
dire_mid_tier1  = [] 
dire_top_tier3 = [] 
dire_top_tier2  = [] 
dire_top_tier1  = [] 

for z in df["tower_status_dire"]:
    element = format(z,"016b")
    if element[5]=='1':
        dire_ancient_bottom.append(1)
    else:
        dire_ancient_bottom.append(0)
    if element[6]=='1':
        dire_ancient_top.append(1)
    else:
        dire_ancient_top.append(0)
    if element[7]=='1':
        dire_bot_tier3.append(1)
    else:
        dire_bot_tier3.append(0)
    if element[8]=='1':
        dire_bot_tier2.append(1)
    else:
        dire_bot_tier2.append(0) 
    if element[9]=='1':
        dire_bot_tier1.append(1)
    else:
        dire_bot_tier1.append(0) 
    if element[10]=='1':
        dire_mid_tier3.append(1)
    else:
        dire_mid_tier3.append(0) 
    if element[11]=='1':
        dire_mid_tier2.append(1)
    else:
        dire_mid_tier2.append(0)
    if element[12]=='1':
        dire_mid_tier1.append(1)
    else:
        dire_mid_tier1.append(0)
    if element[13]=='1':
        dire_top_tier3.append(1)
    else:
        dire_top_tier3.append(0)
    if element[14]=='1':
        dire_top_tier2.append(1)
    else:
        dire_top_tier2.append(0)
    if element[15]=='1':
        dire_top_tier1.append(1)
    else:
        dire_top_tier1.append(0)

df["dire_ancient_bottom"] = dire_ancient_bottom 
df["dire_ancient_top"] = dire_ancient_top 
df["dire_bot_tier3"] = dire_bot_tier3 
df["dire_bot_tier2"] = dire_bot_tier2  
df["dire_bot_tier1"] = dire_bot_tier1   
df["dire_mid_tier3"] = dire_mid_tier3 
df["dire_mid_tier2"] = dire_mid_tier2   
df["dire_mid_tier1"] = dire_mid_tier1  
df["dire_top_tier3"] = dire_top_tier3 
df["dire_top_tier2"] = dire_top_tier2  
df["dire_top_tier1"] = dire_top_tier1  

In [118]:
#dire barracks
dire_bot_ranged = [] 
dire_bot_melee = [] 
dire_mid_ranged = [] 
dire_mid_melee  = [] 
dire_top_ranged  = [] 
dire_top_melee = [] 


for z in df["barracks_status_dire"]:
    element = format(z,"08b")
    if element[2]=='1':
        dire_bot_ranged.append(1)
    else:
        dire_bot_ranged.append(0)
    if element[3]=='1':
        dire_bot_melee.append(1)
    else:
        dire_bot_melee.append(0)
    if element[4]=='1':
        dire_mid_ranged.append(1)
    else:
        dire_mid_ranged.append(0)
    if element[5]=='1':
        dire_mid_melee.append(1)
    else:
        dire_mid_melee.append(0) 
    if element[6]=='1':
        dire_top_ranged.append(1)
    else:
        dire_top_ranged.append(0) 
    if element[7]=='1':
        dire_top_melee.append(1)
    else:
        dire_top_melee.append(0) 

df["dire_bot_ranged"] = dire_bot_ranged 
df["dire_bot_melee"] = dire_bot_melee  
df["dire_mid_ranged"] = dire_mid_ranged 
df["dire_mid_melee"] = dire_mid_melee 
df["dire_top_ranged"] = dire_top_ranged  
df["dire_top_melee"] = dire_top_melee 


In [119]:
#radiant barracks
radiant_bot_ranged = [] 
radiant_bot_melee = [] 
radiant_mid_ranged = [] 
radiant_mid_melee  = [] 
radiant_top_ranged  = [] 
radiant_top_melee = [] 


for z in df["barracks_status_radiant"]:
    element = format(z,"08b")
    if element[2]=='1':
        radiant_bot_ranged.append(1)
    else:
        radiant_bot_ranged.append(0)
    if element[3]=='1':
        radiant_bot_melee.append(1)
    else:
        radiant_bot_melee.append(0)
    if element[4]=='1':
        radiant_mid_ranged.append(1)
    else:
        radiant_mid_ranged.append(0)
    if element[5]=='1':
        radiant_mid_melee.append(1)
    else:
        radiant_mid_melee.append(0) 
    if element[6]=='1':
        radiant_top_ranged.append(1)
    else:
        radiant_top_ranged.append(0) 
    if element[7]=='1':
        radiant_top_melee.append(1)
    else:
        radiant_top_melee.append(0) 

df["radiant_bot_ranged"] = radiant_bot_ranged 
df["radiant_bot_melee"] = radiant_bot_melee  
df["radiant_mid_ranged"] = radiant_mid_ranged 
df["radiant_mid_melee"] = radiant_mid_melee 
df["radiant_top_ranged"] = radiant_top_ranged  
df["radiant_top_melee"] = radiant_top_melee 


In [120]:
#reformating
#some values from this website https://wiki.teamfortress.com/wiki/WebAPI/GetMatchDetails#Tower_Status

rdf = df
#converting radiant win binary 
rdf['radiant_win'] = rdf['radiant_win'].replace('t', 1)
rdf['radiant_win'] = rdf['radiant_win'].replace('f', 0)

#keeping it exclusively ranked matches
rdf = rdf[rdf["lobby_type"] == 7]
rdf = rdf[rdf["game_mode"] == 22]

#making sure there's no "NaN"
rdf = rdf.dropna(subset=['teamfights', 'radiant_gold_adv','radiant_xp_adv'])

rdf = rdf.drop("tower_status_radiant",axis=1)
rdf = rdf.drop("tower_status_dire",axis=1)

print(rdf.shape)
print(rdf)

(3887, 45)
        radiant_win  duration  barracks_status_radiant  barracks_status_dire  \
299               0      1374                        0                    63   
420               1      3208                       63                     3   
421               0      2638                        0                    63   
422               0      2953                        3                    63   
832               1      1956                       63                     3   
1380              0      3082                        0                    63   
2314              0      1723                        0                    63   
2315              0      1320                       48                    63   
2316              1      2914                       55                     3   
2571              0      2325                        0                    63   
2588              1      2016                       63                     0   
2652              0      2154

In [121]:
#for long running cleaning

#replacing gold_adv array to last element of array (what they end up getting at the end of the game)
for x in rdf['radiant_gold_adv']:
    text = x
    text = text.replace('{', '')
    text = text.replace('}', '')
    text = text.split(',')
    element = int(text[-1])
    index = (rdf['radiant_gold_adv']==x).idxmax()
    rdf['radiant_gold_adv'][index] = element

#replacing xp_adv array to last element of array (what they end up getting at the end of the game)
for x in rdf['radiant_xp_adv']:
    text = x
    text = text.replace('{', '')
    text = text.replace('}', '')
    text = text.split(',')
    element = int(text[-1])
    index = (rdf['radiant_xp_adv']==x).idxmax()
    rdf['radiant_xp_adv'][index] = element

C:\Users\aegis\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\aegis\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [122]:
#teamfights might change it to number of teamfights
for x in rdf['teamfights']:
    text = x
    text = text.split('"start')
    element = len(text)
    index = (rdf['teamfights']==x).idxmax()
    rdf['teamfights'][index] = element

C:\Users\aegis\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [124]:
rdf = rdf.drop("lobby_type", axis=1)
rdf = rdf.drop("game_mode", axis=1)
rdf

,radiant_win,duration,barracks_status_radiant,barracks_status_dire,first_blood_time,human_players,radiant_gold_adv,radiant_xp_adv,teamfights,radiant_ancient_bottom,...,dire_mid_ranged,dire_mid_melee,dire_top_ranged,dire_top_melee,radiant_bot_ranged,radiant_bot_melee,radiant_mid_ranged,radiant_mid_melee,radiant_top_ranged,radiant_top_melee
299,0,1374,0,63,45,10,-27097,-12710,4,0,...,1,1,1,1,0,0,0,0,0,0
420,1,3208,63,3,17,10,17736,23617,13,1,...,0,0,1,1,1,1,1,1,1,1
421,0,2638,0,63,0,10,-32423,-28504,14,0,...,1,1,1,1,0,0,0,0,0,0
422,0,2953,3,63,8,10,-22818,-18509,16,0,...,1,1,1,1,0,0,0,0,1,1
832,1,1956,63,3,306,10,32094,39541,13,1,...,0,0,1,1,1,1,1,1,1,1
1380,0,3082,0,63,12,10,-23233,-5860,17,0,...,1,1,1,1,0,0,0,0,0,0
2314,0,1723,0,63,130,10,-40967,-23736,10,0,...,1,1,1,1,0,0,0,0,0,0
2315,0,1320,48,63,113,10,-21398,-16288,6,0,...,1,1,1,1,1,1,0,0,0,0
2316,1,2914,55,3,6,10,14147,21607,16,1,...,0,0,1,1,1,1,0,1,1,1
2571,0,2325,0,63,7,10,-30449,-26200,10,0,...,1,1,1,1,0,0,0,0,0,0


In [2]:
from sklearn.model_selection import KFold

def accuracy(conf_mat):
    return (conf_mat[0, 0] + conf_mat[1, 1])/np.sum(np.ravel(conf_mat))
def error_rate(conf_mat):
    return 1 - accuracy(conf_mat) 

k = 5 
threshold = 0.5
kf = KFold(n_splits=k, shuffle=True)
train_trrs = []
test_trrs = []

for train_index, test_index in kf.split(rdf):
    df_train = rdf.iloc[train_index, :]
    df_test = rdf.iloc[test_index, :]
    
    # logistic regression on training data
    #res = smf.logit('', data=).fit(disp=0)
    res = smf.logit('radiant_win', data=df).fit(disp=0)
    
    # training total error rate 
    train_pred = (res.predict(df_train) > threshold).astype(int)
    train_conf_mat = metrics.confusion_matrix(df_train['Survived'], train_pred)
    train_trr = error_rate(train_conf_mat)
    
    # test total error rate 
    test_pred = (res.predict(df_test) > threshold).astype(int)
    test_conf_mat = metrics.confusion_matrix(df_test['Survived'], test_pred)
    test_trr = error_rate(test_conf_mat)
    
    # record training error and test error
    train_trrs.append(train_trr)
    test_trrs.append(test_trr)
    
# summarize results into a table
res_table = pd.DataFrame()
res_table['training error rate'] = train_trrs
res_table['test error rate'] = test_trrs
res_table


NameError: name 'df' is not defined

In [ ]:
predictors = df.columns.values[1:]

formula = 'BehaviorResponse ~ ' + ' + '.join(predictors)
print(formula)

results = smf.ols(formula, data=df).fit()
results.summary()